In [2]:
#Set the folder paths so that it can be used later. 
bronze_folder_path = 'abfss://bronze@vnpayproject.dfs.core.windows.net/'
silver_folder_path = 'abfss://silver@vnpayproject.dfs.core.windows.net/__unitystorage/schemas/72c947f6-1c6b-4578-9cc6-f65e3f3118a7/tables/'
new_silver_folder_path = 'abfss://silver@vnpayproject.dfs.core.windows.net/'
gold_folder_path = 'abfss://gold@vnpayproject.dfs.core.windows.net/'

StatementMeta(vnpaypoolv1, 2, 3, Finished, Available, Finished)

In [3]:
#Set the spark config to be able to get the partitioned columns year and month as strings rather than integers
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

StatementMeta(vnpaypoolv1, 1, 4, Finished, Available, Finished)

In [4]:
# Read with schema enforcement
all_transaction_df = spark.read.format("delta") \
    .option("mergeSchema", "false") \
    .load(f"{silver_folder_path}/3ef73633-4816-421f-aabf-76ee36b90d6a")

StatementMeta(vnpaypoolv1, 1, 5, Finished, Available, Finished)

In [5]:
display(all_transaction_df.limit(20))

StatementMeta(vnpaypoolv1, 1, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0cd944c5-2b36-49e0-a47f-7433904d4a7f)

In [9]:
from pyspark.sql.functions import col, year, month, date_format

# Now create the columns
all_transaction_df = all_transaction_df \
    .withColumn("year", year(col("thoi_gian_thanh_toan"))) \
    .withColumn("month", date_format(col("thoi_gian_thanh_toan"), "MM"))

StatementMeta(vnpaypoolv1, 1, 10, Finished, Available, Finished)

In [10]:
display(all_transaction_df.limit(20))

StatementMeta(vnpaypoolv1, 1, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6fb346ac-e359-41ea-9880-c66ffe0d56fc)

In [13]:
# Write to Delta Lake with partition by year and month
all_transaction_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save(f"{new_silver_folder_path}/alltransaction_fact")

StatementMeta(vnpaypoolv1, 1, 14, Finished, Available, Finished)

In [14]:
all_transaction_df_new = spark.read.format("delta") \
    .option("mergeSchema", "false") \
    .load(f"{new_silver_folder_path}/alltransaction_fact")

display(all_transaction_df_new.limit(20))

StatementMeta(vnpaypoolv1, 1, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b7205bc0-4a4e-44b3-9803-c7a8564e9493)

In [15]:
all_transaction_df_new.count()

StatementMeta(vnpaypoolv1, 1, 16, Finished, Available, Finished)

87616095

In [16]:
%%sql

-- Create database to which we are going to write the data

CREATE DATABASE IF NOT EXISTS vnpay_silver_spark
LOCATION 'abfss://silver@vnpayproject.dfs.core.windows.net/';

StatementMeta(vnpaypoolv1, 1, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
# Read the Delta Lake file into DataFrame
all_transaction_df_new = spark.read.format("delta") \
    .option("mergeSchema", "false") \
    .load(f"{new_silver_folder_path}/alltransaction_fact")

# Create External Table pointing to existing Delta Lake location
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS vnpay_silver_spark.alltransaction_fact
    USING DELTA
    LOCATION '{new_silver_folder_path}/alltransaction_fact'
""")

StatementMeta(vnpaypoolv1, 2, 7, Finished, Available, Finished)

DataFrame[]